<a href="https://colab.research.google.com/github/kuansheng-yao/Educational-Big-Data-Projects/blob/main/%E4%BD%A0%E6%98%AF%E4%B8%80%E4%BD%8Dpython%E7%A8%8B%E5%BC%8F%E5%B7%A5%E7%A8%8B%E5%B8%AB_%C2%A0%E8%AB%8B%E5%B0%87%E4%BB%A5%E4%B8%8B%E4%BD%BF%E7%94%A8OpenAI%E7%9A%84%E9%83%A8%E5%88%86%E6%94%B9%E6%88%90%E4%BD%BF%E7%94%A8gemini_%E5%AE%89%E8%A3%9D%E6%89%80%E9%9C%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# 安裝所需的 Python 庫
!pip install gspread gradio gemini
!pip install gemini==0.2.1
!pip install -q -U google-generativeai
!pip install gradio


import pandas as pd
import gspread
import requests
from oauth2client.service_account import ServiceAccountCredentials
import gradio as gr

# 進行 Google Colab 的身份驗證
from google.colab import auth
auth.authenticate_user()

import os
import json
import google.generativeai as genai

import requests

# 提示用戶輸入 API Key
api_key = input("請輸入你的 GIMINI API Key: ")

  Using cached gradio-4.38.1-py3-none-any.whl (12.4 MB)
  Using cached gemini-0.30.2-py3-none-any.whl
  Using cached aiofiles-23.2.1-py3-none-any.whl (15 kB)
  Using cached altair-5.3.0-py3-none-any.whl (857 kB)
  Using cached fastapi-0.111.1-py3-none-any.whl (92 kB)
  Using cached ffmpy-0.3.2-py3-none-any.whl
  Using cached gradio_client-1.1.0-py3-none-any.whl (318 kB)
  Using cached httpx-0.27.0-py3-none-any.whl (75 kB)
  Using cached orjson-3.10.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (141 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl (32 kB)
  Using cached python_multipart-0.0.9-py3-none-any.whl (22 kB)
  Using cached ruff-0.5.2-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (10.1 MB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl (15 kB)
  Using cached tomlkit-0.12.0-py3-none-any.whl (37 kB)
  Using cached uvicorn-0.30.1-py3-none-any.whl (62 kB)
  Using cached websockets-11.0.3-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.man

In [4]:
# 使用 google.generativeai 套件設定 API Key
genai.configure(api_key=api_key)

# 使用 google.generativeai 套件建立 Gemini 模型
from google.generativeai import GenerativeModel

model = GenerativeModel('gemini-1.5-flash')
response = model.generate_content('打上你要問的問題')

print(response.text)

請您提出問題吧！我會盡力回答您的問題。 

您可以問我任何您想了解的事情，例如：

* **知識類問題:** 關於歷史、科學、文化、地理等等。
* **生活類問題:** 關於健康、烹飪、旅遊、科技等等。
* **創造性問題:** 關於寫作、繪畫、音樂等等。
* **其他任何問題:** 只要您想問，我都會盡力幫您解答。

請盡情發問！ 



In [6]:
"""
import googleapiclient.discovery
import google.auth
from google.auth.transport.requests import Request
from oauth2client.client import GoogleCredentials
from gspread import authorize

# 確保你已經將 Google 驗證憑證加載到環境中
creds, _ = google.auth.default()
gc = authorize(creds)

# 讀取 Google Spreadsheets 工作表 : record_sheet
record_sheet_id = '1poMhLbeQSrMNnJgTW5zS1bi_hqf6PnHnEi_2aTJ8kRc'
record_sheet_name = 'record'
record_sheet = gc.open_by_key(record_sheet_id).worksheet(record_sheet_name)

# 定義函數，選擇發票 ID 並篩選對應的資料
def choose_invoice_id(df, invoice_id):
    df['Invoice ID'] = df['Invoice ID'].astype(str)  # 將 Invoice ID 欄位轉換為文字格式
    df_filtered = df[df['Invoice ID'] == invoice_id]  # 篩選出符合條件的資料

    if len(df_filtered) > 0:
        invoice_data = df_filtered[['Invoice ID', 'Branch', 'City', 'Customer type', 'Gender', 'Product line', 'Unit price', 'Quantity', 'Tax 5%', 'Total', 'Date', 'Time', 'Payment', 'cogs', 'gross margin percentage', 'gross income', 'Rating']].to_dict(orient='records')[0]
        for key, value in invoice_data.items():
            if pd.isna(value):
                invoice_data[key] = 'nan'

        today = pd.Timestamp('today')  # 取得目前日期
        today_str = today.strftime('%Y-%m-%d')
        invoice_data['Current Date'] = today_str  # 將目前日期加到字典中

        data = pd.DataFrame([invoice_data]).values.tolist()  # 將DataFrame轉換為二維列表
        data = data[0]  # 將列表轉換為一維
        record_sheet.append_row(data)  # 將data導入record sheet

        return invoice_data, data, invoice_id

    else:
        return None, None, invoice_id

# 定義函數，調用 gemini API 並回應問題
def call_gemini(user_question, df, api_key):
    try:
        record_data = df.to_dict(orient='records')
        json_data = json.dumps(record_data)

        # 設定環境變數
        os.environ['GEMINI_API_KEY'] = api_key

        # 初始化 Gemini 客戶端
        gemini.api_key = api_key

        response = gemini.query(
            text=user_question,
            context=json_data,
            model="gemini-3",
      )
        return response['text']
    except Exception as e:
        return f"Error occurred: {e}"

# 定義主函數，用於回應 Gemini 的問題
def main_gemini_response(user_question, file, api_key):
    df = pd.read_csv(file.name)
    return call_gemini(user_question, df, api_key)

# 使用 Gradio 建立介面
with gr.Blocks() as demo:
    with gr.Tab("Ask Gemini"):
        file_input = gr.File(label="Upload CSV File")
        question_input = gr.Textbox(label="Enter your question")
        openai_output = gr.Textbox(label="AI Response")
        ask_gemini_button = gr.Button("Ask")
        ask_gemini_button.click(fn=main_gemini_response, inputs=[question_input, file_input, api_key_input], outputs=openai_output)

demo.launch()
"""

import googleapiclient.discovery
import google.auth
from google.auth.transport.requests import Request
from oauth2client.client import GoogleCredentials
from gspread import authorize

# 確保你已經將 Google 驗證憑證加載到環境中
creds, _ = google.auth.default()
gc = authorize(creds)

# 讀取 Google Spreadsheets 工作表 : record_sheet
try:
    record_sheet_id = '1poMhLbeQSrMNnJgTW5z1bi_hqf6PnHnEi_2aTJ8kRc'
    record_sheet_name = 'record'
    record_sheet = gc.open_by_key(record_sheet_id).worksheet(record_sheet_name)
except gspread.exceptions.SpreadsheetNotFound as e:
    print(f"Error: Spreadsheet not found. Please check the key and permissions.")
    exit(1)

# 定義函數，選擇發票 ID 並篩選對應的資料
def choose_invoice_id(df, invoice_id):
    df['Invoice ID'] = df['Invoice ID'].astype(str)  # 將 Invoice ID 欄位轉換為文字格式
    df_filtered = df[df['Invoice ID'] == invoice_id]  # 篩選出符合條件的資料

    if len(df_filtered) > 0:
        invoice_data = df_filtered[['Invoice ID', 'Branch', 'City', 'Customer type', 'Gender', 'Product line', 'Unit price', 'Quantity', 'Tax 5%', 'Total', 'Date', 'Time', 'Payment', 'cogs', 'gross margin percentage', 'gross income', 'Rating']].to_dict(orient='records')[0]
        for key, value in invoice_data.items():
            if pd.isna(value):
                invoice_data[key] = 'nan'

        today = pd.Timestamp('today')  # 取得目前日期
        today_str = today.strftime('%Y-%m-%d')
        invoice_data['Current Date'] = today_str  # 將目前日期加到字典中

        data = pd.DataFrame([invoice_data]).values.tolist()  # 將DataFrame轉換為二維列表
        data = data[0]  # 將列表轉換為一維
        record_sheet.append_row(data)  # 將data導入record sheet

        return invoice_data, data, invoice_id

    else:
        return None, None, invoice_id

# 定義函數，調用 gemini API 並回應問題
def call_gemini(user_question, df, api_key):
    try:
        record_data = df.to_dict(orient='records')
        json_data = json.dumps(record_data)

        # 設定環境變數
        # os.environ['GEMINI_API_KEY'] = api_key

        # 初始化 Gemini 客戶端
        # gemini.api_key = api_key

        response = gemini.query(
            text=user_question,
            context=json_data,
            model="gemini-3",
        )
        return response['text']
    except Exception as e:
        return f"Error occurred: {e}"

# 定義主函數，用於回應 Gemini 的問題
def main_gemini_response(user_question, file, api_key):
    df = pd.read_csv(file.name)
    return call_gemini(user_question, df, api_key)

# 使用 Gradio 建立介面
with gr.Blocks() as demo:
    with gr.Tab("Ask Gemini"):
        file_input = gr.File(label="Upload CSV File")
        question_input = gr.Textbox(label="Enter your question")
        openai_output = gr.Textbox(label="AI Response")
        ask_gemini_button = gr.Button("Ask")
        ask_gemini_button.click(fn=main_gemini_response, inputs=[question_input, file_input], outputs=openai_output)

demo.launch()


Error: Spreadsheet not found. Please check the key and permissions.


/usr/local/lib/python3.10/dist-packages/gradio/utils.py:984: UserWarning: Expected 3 arguments for function <function main_gemini_response at 0x7d3e0e624f70>, received 2.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/utils.py:988: UserWarning: Expected at least 3 arguments for function <function main_gemini_response at 0x7d3e0e624f70>, received 2.
  warnings.warn(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://bda8f83a00bf4f57cc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


<div class="md-recitation">
  Sources
  <ol>
  <li><a href="https://tealfeed.com/handle-datasets-star-snowflake-vbxjt">https://tealfeed.com/handle-datasets-star-snowflake-vbxjt</a></li>
  </ol>
</div>